In [4]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os

from trackml.dataset import load_event, load_dataset
from trackml.score import score_event

In [151]:
from sklearn.preprocessing import StandardScaler
import hdbscan
from scipy import stats
from tqdm import tqdm
from sklearn.cluster import DBSCAN

class Clusterer(object):
    def __init__(self,eps, rz_scales=[0.65, 0.965, 1.528]):                        
        self.rz_scales=rz_scales
        self.eps = eps
#     def __init__(self,rz_scales=[1., 1., 1.]):                        
#         self.rz_scales=rz_scales
        
    def _eliminate_outliers(self,labels,M):
        norms=np.zeros((len(labels)),np.float32)
        indices=np.zeros((len(labels)),np.float32)
        for i, cluster in tqdm(enumerate(labels),total=len(labels)):
            if cluster == 0:
                continue
            index = np.argwhere(self.clusters==cluster)
            index = np.reshape(index,(index.shape[0]))
            indices[i] = len(index)
            x = M[index]
            norms[i] = self._test_quadric(x)
        threshold1 = np.percentile(norms,90)*5
        threshold2 = 20
        threshold3 = 7
        for i, cluster in enumerate(labels):
            if norms[i] > threshold1 or indices[i] > threshold2 or indices[i] < threshold3:
                self.clusters[self.clusters==cluster]=0            
    def _test_quadric(self,x):
        Z = np.zeros((x.shape[0],10), np.float32)
        Z[:,0] = x[:,0]**2
        Z[:,1] = 2*x[:,0]*x[:,1]
        Z[:,2] = 2*x[:,0]*x[:,2]
        Z[:,3] = 2*x[:,0]
        Z[:,4] = x[:,1]**2
        Z[:,5] = 2*x[:,1]*x[:,2]
        Z[:,6] = 2*x[:,1]
        Z[:,7] = x[:,2]**2
        Z[:,8] = 2*x[:,2]
        Z[:,9] = 1
        v, s, t = np.linalg.svd(Z,full_matrices=False)        
        smallest_index = np.argmin(np.array(s))
        T = np.array(t)
        T = T[smallest_index,:]        
        norm = np.linalg.norm(np.dot(Z,T), ord=2)**2
        return norm

    def _preprocess(self, hits):
        
        x = hits.x.values
        y = hits.y.values
        z = hits.z.values

        r = np.sqrt(x**2 + y**2 + z**2)
        hits['x2'] = x/r
        hits['y2'] = y/r

        r = np.sqrt(x**2 + y**2)
        hits['z2'] = z/r

        ss = StandardScaler()
        X = ss.fit_transform(hits[['x2', 'y2', 'z2']].values)
        for i, rz_scale in enumerate(self.rz_scales):
            X[:,i] = X[:,i] * rz_scale
       
        return X
    def _init(self, dfh):
        
        dfh['r'] = np.sqrt(dfh.x**2+dfh.y**2+dfh.z**2)
        dfh['rt'] = np.sqrt(dfh.x**2+dfh.y**2)
        dfh['a0'] = np.arctan2(dfh.y,dfh.x)
        dfh['a0_1'] = np.arctan2(dfh.y2,dfh.x2)
        dfh['r2'] = np.sqrt(dfh.x**2+dfh.y**2)
        dfh['z1'] = dfh['z']/dfh['rt'] 
        
        x = dfh.x.values
        y = dfh.y.values
        z = dfh.z.values
        
        x2 = dfh.x2.values
        y2 = dfh.y2.values
        z2 = dfh.z2.values
        dfh['rho'] = np.sqrt(x**2 + y**2 + z**2)
        dfh['r'] = np.sqrt(x**2 + y**2)
        dfh['r2'] = np.sqrt(x2**2 + y2**2)
#         dfh['phi'] = np.degrees(np.arctan2(hits['y2'], hits['x2']))
        dfh['phi'] = np.arctan2(dfh['y'], dfh['x'])
        dfh['phi2'] = np.arctan2(dfh['y2'], dfh['x2'])
#         dfh['phi_1'] = np.round(np.degrees(np.arctan2(hits['y2'], hits['x2'])), 5)
#         dfh['theta'] = np.degrees(np.arctan2(hits['r'], hits['z2']))
        dfh['theta'] = np.arctan2(dfh['r'], dfh['z'])
        phi = dfh['phi'].values
        theta = dfh['theta'].values
        rho = dfh['rho'].values
        r = dfh['r'].values
        
        phi = dfh['phi2'].values
        r = dfh['r2'].values
        
        dfh['tan_dip'] = phi/theta
        dfh['tan_dip1'] = phi/z2
        dfh['z2_1'] = 1/z2
        dfh['z2_2'] = phi/z2 + 1/z2

        dz = 0.00012
        stepdz = 0.000005
        for ii in tqdm(range(24)):
            dz = dz + ii*stepdz
            dfh['a1'] = dfh['a0']+dz*dfh['z']*np.sign(dfh['z'].values)
            dfh['x1'] = dfh['a1']/dfh['z1']
            dfh['x2'] = 1/dfh['z1']
            dfh['x3'] = dfh['x1']+dfh['x2']
            
            dfh['z2_1'] = dfh['a1']/z2
            dfh['z2_2'] = 1/z2
            dfh['z2_3'] =  dfh['z2_1'] + dfh['z2_2']
            dz1 = 0.0012
            stepdz1 = 0.00005
            dz1 = dz1 + ii*stepdz1
            ss = StandardScaler()
            dfs = ss.fit_transform(dfh[['a1','z1','x1','x2','x3']].values)
#             dfs = ss.fit_transform(dfh[['a1','z2','z2_1', 'z2_2', 'z2_3']].values)
#             dfs = ss.fit_transform(dfh[['a1', 'z2', 'phi2', 'r2', 'z2_1', 'z2_2', 'z2_3']].values)
#             'z2', 'phi2', 'r2','tan_dip1', 'z2_1', 'z2_2'
#             clusters = DBSCAN(eps=self.eps-dz1,min_samples=1,metric='manhattan',n_jobs=8).fit(dfs).labels_
            
            clusters = DBSCAN(eps=0.0035-dz,min_samples=1,metric='manhattan',n_jobs=8).fit(dfs).labels_
#             clusters = DBSCAN(self.eps-dz,min_samples=1,metric='manhattan',n_jobs=8).fit(dfs).labels_
            if ii==0:
                dfh['s1']= clusters
                dfh['N1'] = dfh.groupby('s1')['s1'].transform('count')
            else:
                dfh['s2'] = clusters
                dfh['N2'] = dfh.groupby('s2')['s2'].transform('count')
                maxs1 = dfh['s1'].max()
                cond = np.where(dfh['N2'].values>dfh['N1'].values)
                s1 = dfh['s1'].values
                s1[cond] = dfh['s2'].values[cond]+maxs1
                dfh['s1'] = s1
                dfh['s1'] = dfh['s1'].astype('int64')
                self.clusters = dfh['s1'].values
                dfh['N1'] = dfh.groupby('s1')['s1'].transform('count')
        dz = 0.00012
        stepdz = -0.000005
        for ii in tqdm(range(24)):
            dz = dz + ii*stepdz
            dfh['a1'] = dfh['a0']+dz*dfh['z']*np.sign(dfh['z'].values)
            dfh['x1'] = dfh['a1']/dfh['z1']
            dfh['x2'] = 1/dfh['z1']
            dfh['x3'] = dfh['x1']+dfh['x2']
            
            # vary phi2
            dfh['a2'] = dfh['phi2']+dz*dfh['z']*np.sign(dfh['z'].values)
            dfh['z2_1'] = dfh['a2']/z2 # dip angle?
            dfh['z2_2'] = 1/z2
            dfh['z2_3'] =  dfh['z2_1'] + dfh['z2_2']
            
            dz1 = 0.0012
            stepdz1 = 0.00005
            dz1 = dz1 + ii*stepdz1
            ss = StandardScaler()
            dfs = ss.fit_transform(dfh[['a1','z1','x1','x2','x3']].values)
#             dfs = ss.fit_transform(dfh[['a1', 'z2', 'phi2', 'r2', 'z2_1', 'z2_2', 'z2_3']].values)
#             dfs = ss.fit_transform(dfh[['a1','z2','z2_1', 'z2_2', 'z2_3']].values)
            clusters = DBSCAN(eps=0.0035+dz,min_samples=1,metric='manhattan',n_jobs=8).fit(dfs).labels_
#             clusters = DBSCAN(eps=self.eps+dz1,min_samples=1,metric='manhattan',n_jobs=8).fit(dfs).labels_
#             clusters = DBSCAN(self.eps+dz,min_samples=1,metric='manhattan',n_jobs=8).fit(dfs).labels_
#             'z2', 'phi2', 'r2','tan_dip1', 'z2_1', 'z2_2'
#             dfs = ss.fit_transform(dfh[['a1','z2','r2','phi2','tan_dip1', 'z2_1', 'z2_2']].values)
#             clusters = DBSCAN(eps=0.011+dz,min_samples=1,metric='mahattan',n_jobs=8).fit(dfs).labels_
            
            dfh['s2'] = clusters
            dfh['N2'] = dfh.groupby('s2')['s2'].transform('count')
            maxs1 = dfh['s1'].max()
            cond = np.where(dfh['N2'].values>dfh['N1'].values)
            s1 = dfh['s1'].values
            s1[cond] = dfh['s2'].values[cond]+maxs1
            dfh['s1'] = s1
            dfh['s1'] = dfh['s1'].astype('int64')
            dfh['N1'] = dfh.groupby('s1')['s1'].transform('count')
            
            # check
            dfh['track_id'] = dfh['s1'].values
            dfh['event_id'] = 0
            print(score_event(dfh, dfh[['event_id', 'hit_id', 'track_id']]))
        return dfh['s1'].values
    def predict(self, hits):  
        X = self._preprocess(hits)
        self.clusters = self._init(hits)                
        
        cl = hdbscan.HDBSCAN(min_samples=1,min_cluster_size=7,
                             metric='braycurtis',cluster_selection_method='leaf',algorithm='best', 
                             leaf_size=50)
        labels = np.unique(self.clusters)
        n_labels = 0
        while n_labels < len(labels):
            n_labels = len(labels)
            self._eliminate_outliers(labels,X)
            max_len = np.max(self.clusters)
            self.clusters[self.clusters==0] = cl.fit_predict(X[self.clusters==0])+max_len
            labels = np.unique(self.clusters)
        return self.clusters

In [49]:
def create_one_event_submission(event_id, hits, labels):
    sub_data = np.column_stack(([event_id]*len(hits), hits.hit_id.values, labels))
    submission = pd.DataFrame(data=sub_data, columns=["event_id", "hit_id", "track_id"]).astype(int)
    return submission

In [82]:
path_to_train = '../data/train'
event_prefix = "event000001000"
hits, cells, particles, truth = load_event(os.path.join(path_to_train, event_prefix))

In [61]:
model = Clusterer()
labels = model.predict(hits)

 42%|████▏     | 10/24 [00:08<00:12,  1.12it/s]


KeyboardInterrupt: 

In [11]:
submission = create_one_event_submission(0, hits, labels)
score = score_event(truth, submission)
print("Your score: ", score)

Your score:  0.3724995097633181


In [127]:
model = Clusterer()
labels = model.predict(hits)
submission = create_one_event_submission(0, hits, labels)
score = score_event(truth, submission)
print("Your score: ", score)

100%|██████████| 46651/46651 [00:07<00:00, 6261.28it/s]


Your score:  0.33537467511461244


In [133]:
for eps in [0.0025, 0.003, 0.0035, 0.004, 0.0045]:
    model = Clusterer(eps)
    labels = model.predict(hits)
    submission = create_one_event_submission(0, hits, labels)
    score = score_event(truth, submission)
    print("Your score: ", score)

  0%|          | 0/24 [00:00<?, ?it/s]

Your score:  0.3316338683211663


  0%|          | 0/24 [00:00<?, ?it/s]

Your score:  0.35494782255692414


  0%|          | 0/24 [00:00<?, ?it/s]

Your score:  0.3724995097633181


  0%|          | 0/24 [00:00<?, ?it/s]

Your score:  0.3678402221992445


100%|██████████| 37930/37930 [00:06<00:00, 6316.72it/s]


Your score:  0.35777253808781195


In [135]:
for eps in [0.009, 0.01, 0.015]:
    model = Clusterer(eps)
    labels = model.predict(hits)
    submission = create_one_event_submission(0, hits, labels)
    score = score_event(truth, submission)
    print("Your score: ", score)

  0%|          | 0/24 [00:00<?, ?it/s]

Your score:  0.24455597857829214


  0%|          | 0/24 [00:00<?, ?it/s]

Your score:  0.24760303721305957


100%|██████████| 57403/57403 [00:08<00:00, 6614.96it/s]


Your score:  0.26580657537774616


In [136]:
for eps in [0.015, 0.02, 0.025, 0.03, 0.035]:
    model = Clusterer(eps)
    labels = model.predict(hits)
    submission = create_one_event_submission(0, hits, labels)
    score = score_event(truth, submission)
    print("Your score: ", score)

  0%|          | 0/24 [00:00<?, ?it/s]

Your score:  0.26580657537774616


  0%|          | 0/24 [00:00<?, ?it/s]

Your score:  0.2779047770395823


  0%|          | 0/24 [00:00<?, ?it/s]

Your score:  0.2731438906619206


  0%|          | 0/24 [00:00<?, ?it/s]

Your score:  0.2510339295763603


100%|██████████| 10181/10181 [00:01<00:00, 5800.38it/s]


Your score:  0.2478649934517582


In [137]:
for eps in [0.02, 0.021, 0.022, 0.023, 0.024, 0.025]:
    model = Clusterer(eps)
    labels = model.predict(hits)
    submission = create_one_event_submission(0, hits, labels)
    score = score_event(truth, submission)
    print("Your score: ", score)

  0%|          | 0/24 [00:00<?, ?it/s]

Your score:  0.2779047770395823


  0%|          | 0/24 [00:00<?, ?it/s]

Your score:  0.27658018631173864


  0%|          | 0/24 [00:00<?, ?it/s]

...

Your score:  0.27508225396290126


  0%|          | 0/24 [00:00<?, ?it/s]

Your score:  0.27420398561069526


100%|██████████| 28283/28283 [00:04<00:00, 6279.68it/s]


Your score:  0.2731438906619206


In [138]:
for eps in [0.0195, 0.0196, 0.0197, 0.0198, 0.0199, 0.02, 0.0221]:
    model = Clusterer(eps)
    labels = model.predict(hits)
    submission = create_one_event_submission(0, hits, labels)
    score = score_event(truth, submission)
    print("Your score: ", score)

  0%|          | 0/24 [00:00<?, ?it/s]

Your score:  0.27899005323946957


  0%|          | 0/24 [00:00<?, ?it/s]

Your score:  0.2783438965105194


  0%|          | 0/24 [00:00<?, ?it/s]

...

Your score:  0.27838525377865697


  0%|          | 0/24 [00:00<?, ?it/s]

Your score:  0.2779047770395823


100%|██████████| 35484/35484 [00:05<00:00, 6026.46it/s]


Your score:  0.2779769871040115


In [139]:
for eps in [0.0190, 0.0191, 0.0192, 0.0193, 0.0194, 0.0195]:
    model = Clusterer(eps)
    labels = model.predict(hits)
    submission = create_one_event_submission(0, hits, labels)
    score = score_event(truth, submission)
    print("Your score: ", score)

  0%|          | 0/24 [00:00<?, ?it/s]

Your score:  0.2784253653607127


  0%|          | 0/24 [00:00<?, ?it/s]

Your score:  0.27790839642284904


  0%|          | 0/24 [00:00<?, ?it/s]

...

Your score:  0.2786410817632458


  0%|          | 0/24 [00:00<?, ?it/s]

Your score:  0.2794540779419116


100%|██████████| 42818/42818 [00:06<00:00, 6361.96it/s]


Your score:  0.27899005323946957


In [140]:
for eps in [0.01940, 0.01941, 0.01942, 0.01943, 0.01944, 0.01945]:
    model = Clusterer(eps)
    labels = model.predict(hits)
    submission = create_one_event_submission(0, hits, labels)
    score = score_event(truth, submission)
    print("Your score: ", score)

  0%|          | 0/24 [00:00<?, ?it/s]

Your score:  0.2794540779419116


  0%|          | 0/24 [00:00<?, ?it/s]

Your score:  0.2796902112536624


  0%|          | 0/24 [00:00<?, ?it/s]

...

Your score:  0.27919318824042855


  0%|          | 0/24 [00:00<?, ?it/s]

Your score:  0.27926378008881425


100%|██████████| 42979/42979 [00:06<00:00, 6473.17it/s]


Your score:  0.27910077055062


In [141]:
for eps in [0.01941, 0.019412, 0.019413, 0.019414, 0.019415]:
    model = Clusterer(eps)
    labels = model.predict(hits)
    submission = create_one_event_submission(0, hits, labels)
    score = score_event(truth, submission)
    print("Your score: ", score)

  0%|          | 0/24 [00:00<?, ?it/s]

Your score:  0.2796902112536624


  0%|          | 0/24 [00:00<?, ?it/s]

Your score:  0.279684001582719


  0%|          | 0/24 [00:00<?, ?it/s]

Your score:  0.2795959374951502


  0%|          | 0/24 [00:00<?, ?it/s]

Your score:  0.27949082549908844


100%|██████████| 43079/43079 [00:06<00:00, 6479.18it/s]


Your score:  0.27949082549908844


In [152]:
path_to_train = '../data/train'
event_prefix = "event000001000"
hits, cells, particles, truth = load_event(os.path.join(path_to_train, event_prefix))

truth = pd.merge(truth, particles, how='left', on='particle_id')
hits = pd.merge(hits, truth, how='left', on='hit_id')

hits = hits.fillna(0)
    
model = Clusterer(eps=0.0035)
labels = model.predict(hits)
submission = create_one_event_submission(0, hits, labels)
score = score_event(truth, submission)
print("Your score: ", score)

  4%|▍         | 1/24 [00:01<00:42,  1.83s/it]

0.0752305681290277


  8%|▊         | 2/24 [00:03<00:40,  1.86s/it]

0.07860727762759465


 12%|█▎        | 3/24 [00:05<00:39,  1.86s/it]

...

0.3393908779413055


  1%|          | 481/44451 [00:00<00:09, 4806.48it/s]

0.34013065376921303


100%|██████████| 44451/44451 [00:07<00:00, 6047.93it/s]


Your score:  0.3724995097633181
